# Getting the weather data of the airports

Using the table at http://www.flugzeuginfo.net/table_airportcodes_country-location_en.php
a csv with the IATA and ICAO codes of the airports can be created.

Using the ICAO codes the weather data can be retrieved from this website: https://www.wunderground.com/history/daily/de/frankfurt/EDDF/date/2015-3-18

Given a list with the destination airport codes and the schedules arrival time, a list of the corresponding weather conditions (wind speed, visibility) can be web scraped.

This list of weater conditions can then be incorporated into the dataframe with the delays and be subsequently used in a predictive model.

In [1]:
import pandas as pd

In [78]:
# The airport and date and time of departure
df_dep = pd.read_csv("data/sanitized_Train_data.csv", usecols=[3, 5], parse_dates=[1])
df_dep.columns = ['IATA', 'DATE_TIME']

In [79]:
df_dep.head()

,IATA,DATE_TIME
0,CMN,2016-01-03 10:30:00
1,MXP,2016-01-13 15:05:00
2,TUN,2016-01-16 04:10:00
3,DJE,2016-01-17 14:10:00
4,TUN,2016-01-17 14:30:00


In [80]:
# The airport and date and time of departure
df_dest = pd.read_csv("data/sanitized_Train_data.csv", usecols=[4, 6], parse_dates=[1])
df_dest.columns = ['IATA', 'DATE_TIME']

In [81]:
df_dest.head()

,IATA,DATE_TIME
0,TUN,2016-01-03 12:55:00
1,TUN,2016-01-13 16:55:00
2,IST,2016-01-16 06:45:00
3,NTE,2016-01-17 17:00:00
4,ALG,2016-01-17 15:50:00


In [82]:
df = pd.DataFrame()
df = pd.concat([df_dep, df_dest], axis=0)
df

,IATA,DATE_TIME
0,CMN,2016-01-03 10:30:00
1,MXP,2016-01-13 15:05:00
2,TUN,2016-01-16 04:10:00
3,DJE,2016-01-17 14:10:00
4,TUN,2016-01-17 14:30:00
...,...,...
107828,TUN,2018-07-06 02:00:00
107829,TUN,2018-01-13 09:00:00
107830,TUN,2018-11-07 12:50:00
107831,DJE,2018-01-23 18:45:00


In [83]:
df_airport_codes = pd.read_csv('list_IATA_ICAO_codes.csv', usecols=[0, 1])

In [84]:
code = df_airport_codes[df_airport_codes['IATA'] == 'CMN']['ICAO']
code = code.iloc[0]
type(code)

str

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215666 entries, 0 to 107832
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   IATA       215666 non-null  object        
 1   DATE_TIME  215666 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 4.9+ MB


In [86]:
df = df.join(df_airport_codes.set_index('IATA'), on='IATA', how='left', lsuffix='_left', rsuffix='_right')

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215666 entries, 0 to 107832
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   IATA       215666 non-null  object        
 1   DATE_TIME  215666 non-null  datetime64[ns]
 2   ICAO       212380 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 6.6+ MB


### Now the weather data have to be retrieved from the website using some web scraper

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [18]:
import time

from selenium import webdriver



driver = webdriver.Chrome(executable_path='/home/fklein/Downloads/chromedriver')  # Optional argument, if not specified will search path.

display('before sleep')
driver.get('http://www.google.com/');

# time.sleep(5) # Let the user actually see something!

# search_box = driver.find_element_by_name('q')

# search_box.send_keys('ChromeDriver')

# search_box.submit()

# time.sleep(5) # Let the user actually see something!

driver.quit()

/tmp/ipykernel_131228/2439431830.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/home/fklein/Downloads/chromedriver')  # Optional argument, if not specified will search path.


WebDriverException: Message: unknown error: Chrome failed to start: exited normally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /snap/bin/chromium is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x56335242b7d3 <unknown>
#1 0x563352187688 <unknown>
#2 0x5633521aab27 <unknown>
#3 0x5633521a668f <unknown>
#4 0x5633521e1355 <unknown>
#5 0x5633521db5a3 <unknown>
#6 0x5633521b0ddc <unknown>
#7 0x5633521b1de5 <unknown>
#8 0x56335245c49d <unknown>
#9 0x56335247560c <unknown>
#10 0x56335245e205 <unknown>
#11 0x563352475ee5 <unknown>
#12 0x563352452070 <unknown>
#13 0x563352491488 <unknown>
#14 0x56335249160c <unknown>
#15 0x5633524aac6d <unknown>
#16 0x7fe32acc0947 <unknown>
